# Maximum Likelihood Estimation

Maximum Likelihood Estimation is a method of calculating the latent parameter/s of a model given a set of observations.

It is so called as it attempts to find the parameter values that are most likely to have produced the observed data—maximises the likelihood function.

The technique may be used in practice when it is infeasible to observe every member of a population and thus parameter estimates from a sample must be made.

## Formulation

Given a set of n i.i.d observations $x_1, x_2,......x_n$ which share a common distribution $p(\cdot|\theta)$ where theta is a vector of parameters for this family. Our goal is to find an estimator $\hat\theta$ that best describes true value of $\theta$.

We may reformulate this problem by specifying the joint density of all the observations.

This is given as $ p(x_1|\theta) \times p(x_2|\theta) ... \times p(x_n|\theta) = p(x_1,x_2,...,x_n|\theta)$ which in turn decomposes to  $\prod_{i=1}^n p(x_i | \theta)$

We now need to find theta. Maximum likelihood seeks the find value of theta that maximizes the probability of our joint probability function $p(x_1,x_2,...,x_n|\theta)$. The problem can in fact be rephrased as finding the value of $\theta$ that most likely given the observed data, which can be expressed mathematically as $L = (\theta | x_1,x_2,...,x_n)$.

The likelihood function is maximised when $$\nabla \prod_{i=1}^n p(x_i | \theta) = 0$$

In practice it is often easier to work with the natural logarithm of the likelihood and so we seek to find 